Implement a transformer architecture and improve and analyze it's arous aspects rather than focus on a particular paper

Use percentage changes as data source instead of prices

If we make a predictor which just guesses next day's price as the same as today’s price, it would have better than 95% accuracy. 

Guessing whether next day price will go up or down i.e. as a classification problem.

In [ ]:
#W&B Setup
!pip install wandb
import wandb

In [7]:
from google.colab import drive
import numpy as np
import pandas as pd
import torch
from torchsummary import summary
drive.mount("/content/drive")
%cd '/content/drive/MyDrive/WB_Projects/Demo/'  

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/WB_Projects/Demo


In [3]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Apr 27 01:13:12 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
#Increase RAM Usage
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [8]:
from utils.get_dataset import GetDataset, StockData
from torch.utils.data import DataLoader

In [25]:
def build_dataset(num_days, batch_size):

  NUM_DAYS = num_days
  KEEP_CLOSE_ONLY = False # debug to drop other columns

  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

  csv = '../data/SPXDailyData.csv'
  df = GetDataset(csv)
  dataset = df.get_data()

  #split into 3
  valid_frac, test_frac = 0.2, 0.2
  train_sz=int(dataset.shape[0]*(1-(valid_frac+test_frac)))
  valid_sz=int(dataset.shape[0]*(valid_frac))
  df_train = dataset[               0:train_sz]
  df_valid = dataset[        train_sz:train_sz+valid_sz]
  df_test = dataset[train_sz+valid_sz:]

  if KEEP_CLOSE_ONLY:# see in case additional info acts like a noise
    df_train.drop(columns=['Open', 'High', 'Low'], inplace=True)
    df_valid.drop(columns=['Open', 'High', 'Low'], inplace=True)
    df_test.drop(columns=['Open', 'High', 'Low'], inplace=True)

  train_dataset = StockData(df_train.to_numpy(), num_days=NUM_DAYS)
  valid_dataset = StockData(df_valid.to_numpy(), num_days=NUM_DAYS) 
  test_dataset = StockData(df_test.to_numpy(), num_days=NUM_DAYS)
 
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
  valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False) 

  return train_loader, valid_loader,  test_dataset

## Training

In [17]:
from utils.utils import plot_curves
from utils.utils import train
from utils.utils import evaluate
from sklearn.exceptions import UndefinedMetricWarning
import math
import copy

In [18]:
def warn(*args, **kwargs):
    pass
import warnings

warnings.warn = warn

In [19]:
#Import Model
from models.my_transformer import TransformerModelImpl

In [26]:
def train_model(config=None): #Configs 
    with wandb.init(config=config): #https://docs.wandb.ai/guides/track/launch
      # This config will be set by Sweep Controller # https://docs.wandb.ai/guides/sweeps/quickstart
      config = wandb.config

      #Generate Train/Validation Loaders and Test Dataset
      train_loader, valid_loader, test_dataset = build_dataset(config.n_days, config.batch_size)

      #Initialize Model
      model = TransformerModelImpl(config)

      #Set optimizer and loss function
      optimizer = torch.optim.Adam(model.parameters(), lr=config.lr)
      criterion = torch.nn.BCEWithLogitsLoss(reduction='mean')

      device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
      model.float()
      model.to(device) #Set model to utilizie available device

      #Dataframes for tracking results
      avg_train_loss,avg_train_acc,avg_valid_loss,avg_valid_acc=[],[],[],[]

      for epoch in range(config.epochs):
          train_loss, atl, ata = train(model, train_loader, optimizer, criterion, device)
          #scheduler.step(train_loss)
          _, avl, ava = evaluate(model, valid_loader, criterion, device)
          if epoch%50==1:
            print("Epoch %d: Training Loss: %.4f. Training Acc: %.4f. Validation Loss: %.4f. Validation Acc: %.4f." % (epoch+1, atl, ata, avl, ava))
          avg_train_loss.append(atl.item())
          avg_train_acc.append(ata)
          avg_valid_loss.append(avl.item())
          avg_valid_acc.append(ava)
          #Weights and biases logging: Reference: https://docs.wandb.ai/guides/track/log
          wandb.log({"train-loss": atl, "epoch": epoch})
          wandb.log({"validation-loss": avl, "epoch": epoch})
          wandb.log({"train-acc":ata, "epoch": epoch})
          wandb.log({"validation-acc": ava, "epoch": epoch})

      #Plot training/loss curves             
      plot_curves(avg_train_loss,avg_train_acc,avg_valid_loss,avg_valid_acc, info='', save=False)   
      #mfinal = copy.deepcopy(model)
    

In [29]:
#Configure Sweep Parameters
#https://docs.wandb.ai/guides/sweeps/quickstart#2.-configure-your-sweep
sweep_config = {
    'method': 'bayes', 
    'metric': {'goal': 'minimize', 'name': 'train-loss'},
    'early_terminate': { 'type': 'hyperband', 's': 2, 'eta': 3,'max_iter': 27}
    }

parameters_dict = {
    'device': {
       'values': torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    },
    'n_days' :{
        'values': [5,10]
    },
     'n_layers': {
        'distribution': 'int_uniform',
        'min': 1,
        'max': 16
    },
    'num_heads': {
        'distribution': 'int_uniform',
        'min': 1,
        'max':6
    },
    'forward_dim': {
      'distribution': 'int_uniform',
      'min': 2,
      'max': 16,
    },
    'output_dim':{
        'values': [1] 
    },
    'model_dim': {
        'values': [4]
    },
    'dropout': {
        'values':[0.1,0.2,0.3]
    },
    'lr': {
        'distribution': 'uniform',
        'min': 0,
        'max': 0.1
    },    
    'epochs' :{
      'distribution': 'int_uniform',
      'min': 5,
      'max': 30,      
    },
    'batch_size':{
      'distribution': 'int_uniform',
      'min': 4,
      'max': 16,      
    },

}

sweep_config['parameters'] = parameters_dict #Set Configs

In [30]:
wandb_project_name = "FinanceTransformer-R1" #User Choice

#Initialize a hyperparameter sweep. 
#https://docs.wandb.ai/ref/python/sweep
sweep_id = wandb.sweep(sweep_config, project = wandb_project_name)

#Run Client
#https://docs.wandb.ai/ref/python/agent
wandb.agent(sweep_id, train_model)

Create sweep with ID: ycienwz9
Sweep URL: https://wandb.ai/mohammadbakir/FinanceTransformer-R1/sweeps/ycienwz9


wandb: Agent Starting Run: arbghcas with config:
wandb: 	batch_size: 14
wandb: 	device: cuda
wandb: 	dropout: 0.2
wandb: 	epochs: 12
wandb: 	forward_dim: 13
wandb: 	lr: 0.013765332185026914
wandb: 	model_dim: 4
wandb: 	n_days: 5
wandb: 	n_layers: 4
wandb: 	num_heads: 2
wandb: 	output_dim: 1


wandb: Ctrl + C detected. Stopping sweep.


## References

Great Youtube Video On Basic Usage & Walk through Colab Link

*   https://www.youtube.com/watch?v=G7GH0SeNBMA
*   https://colab.research.google.com/github/wandb/examples/blob/master/colabs/pytorch/Simple_PyTorch_Integration.ipynb
